In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from datetime import datetime
import pandas as pd
import time
import random
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import warnings
warnings.filterwarnings("ignore")
from socket import *

In [2]:
df = pd.read_csv('stock_code.csv')
df

,stock_company,stock_code
0,7-ELEVEN MALAYSIA HOLDINGS BERHAD,5250.KL
1,ABF MALAYSIA BOND INDEX FUND,0800EA.KL
2,ABLE GLOBAL BERHAD,7167.KL
3,ABLEGROUP BERHAD,7086.KL
4,ABM FUJIYA BERHAD,5198.KL
...,...,...
985,YX PRECIOUS METALS BHD,0250.KL
986,ZECON BERHAD,7028.KL
987,ZELAN BERHAD,2283.KL
988,ZEN TECH INTERNATIONAL BERHAD,0094.KL


In [3]:
df.shape

(990, 2)

In [4]:
df['stock_code']=df['stock_code'].str.slice(0,-3)

In [5]:
stock_code=list(df['stock_code'])
stock_code

['5250',
 '0800EA',
 '7167',
 '7086',
 '5198',
 '03028',
 '7131',
 '0218',
 '0122',
 '1481',
 '5281',
 '9148',
 '7191',
 '7146',
 '0181',
 '6599',
 '5139',
 '5185',
 '7145',
 '7315',
 '7078',
 '0209',
 '5238',
 '2658',
 '7609',
 '5116',
 '5115',
 '2674',
 '0079',
 '2488',
 '1163',
 '03051',
 '5269',
 '5127',
 '5293',
 '5120',
 '03011',
 '1015',
 '7031',
 '6351',
 '7083',
 '0048',
 '4758',
 '0226',
 '6556',
 '5082',
 '9342',
 '5568',
 '5088',
 '7090',
 '5015',
 '6432',
 '0119',
 '7214',
 '7181',
 '7007',
 '0038',
 '0068',
 '7722',
 '7129',
 '0159',
 '0105',
 '4057',
 '7162',
 '7054',
 '03032',
 '6399',
 '0072',
 '8176',
 '7048',
 '5130',
 '7099',
 '5302',
 '03037',
 '8885',
 '5204',
 '7579',
 '6888',
 '5106',
 '7120',
 '2305',
 '5021',
 '7005',
 '03012',
 '0098',
 '5258',
 '7251',
 '1899',
 '6602',
 '0187',
 '5190',
 '3239',
 '3395',
 '5196',
 '4219',
 '5248',
 '9814',
 '7668',
 '6173',
 '5932',
 '0195',
 '6998',
 '5032',
 '0179',
 '5069',
 '0168',
 '9288',
 '7036',
 '8133',
 '2771',
 '

In [6]:
div_info_final=pd.DataFrame()
driver = webdriver.Chrome()
driver.implicitly_wait(1209600) #maximum time to load the link

#stock_c=['7146']
new_list=[]
for s in stock_code:
    try: 
        url='https://www.klsescreener.com/v2/stocks/view/'+s
        driver.get(url)
        p= driver.find_element(By.XPATH,'//*[@id="myTab"]/li[3]/a')
        driver.execute_script("arguments[0].click();",p)
        content = driver.page_source
        soup=bs(content)

        # scrap table
        try:
            tables = soup.find_all('table')
            # Step 3: Read tables with Pandas read_html()
            dfs = pd.read_html(str(tables))

            div_info2=dfs[6]     
            #print(div_info2)

        except:
            print('Stock code: '+s+' cannot scrap')



        #change data types 
        try:
            div_info2['Announced'] = pd.to_datetime(div_info2['Announced'])
            div_info2['Announced']= pd.to_datetime(div_info2['Announced'].dt.strftime('%Y-%m-%d'))

            div_info2['Financial Year'] = pd.to_datetime(div_info2['Financial Year'])
            div_info2['Financial Year']= pd.to_datetime(div_info2['Financial Year'].dt.strftime('%Y-%m-%d'))

            div_info2['EX Date'] = pd.to_datetime(div_info2['EX Date'])
            div_info2['EX Date']= pd.to_datetime(div_info2['EX Date'].dt.strftime('%Y-%m-%d'))

            div_info2['Payment Date'] = pd.to_datetime(div_info2['Payment Date'])
            div_info2['Payment Date']= pd.to_datetime(div_info2['Payment Date'].dt.strftime('%Y-%m-%d'))    
        except:
            print('Stock Code:'+s+' Error in changing datatype date')



        # Filter data between two dates
        try:
            filtered_df = div_info2.loc[(div_info2['Financial Year'] >= '2018-01-01')
                         & (div_info2['Financial Year'] < datetime.now())]

        except:
            print('Stock Code:'+s+' Cannot filter date')


        # check whether the filtered_df is empty
        if filtered_df.empty:
             print('Stock Code:'+s+' has no dividend info')

        # add stock code and year
        try:
            print(s)
            filtered_df['Stock Code']= s
            filtered_df['Year']=filtered_df['Financial Year'].dt.strftime("%Y")
        except:
            print('Stock Code:'+s+' Error in adding columns')

        try:
            del filtered_df['Unnamed: 7']
        except:
             print('Stock Code:'+s+' cannot delete columns Unnamed: 7')
        div_info_final = div_info_final.append(filtered_df, ignore_index = True)

        # Program to generate a random number between 5 to 30
        # importing the random module
        sec = random.randint(5,30)

        # program scrapping waiting time
        time.sleep(sec) # Seconds
        
    except socket.Timeouterror:
        new_list.append(s)

5250
0800EA
7167
Stock Code:7086 has no dividend info
7086
Stock Code:5198 has no dividend info
5198
03028
Stock Code:7131 has no dividend info
7131
0218
Stock Code:0122 has no dividend info
0122
1481
5281
9148
7191
Stock Code:7146 Error in changing datatype date
Stock Code:7146 Cannot filter date
7146
Stock Code:7146 cannot delete columns Unnamed: 7
0181
6599
5139
5185
Stock Code:7145 has no dividend info
7145
Stock Code:7315 has no dividend info
7315
Stock Code:7078 Error in changing datatype date
Stock Code:7078 Cannot filter date
Stock Code:7078 has no dividend info
7078
Stock Code:7078 cannot delete columns Unnamed: 7
0209
Stock Code:5238 has no dividend info
5238
2658
Stock Code:7609 has no dividend info
7609
5116
Stock Code:5115 has no dividend info
5115
2674
Stock Code:0079 has no dividend info
0079
2488
1163
03051
5269
5127
5293
5120
03011
1015
Stock Code:7031 has no dividend info
7031
6351
7083
Stock Code:0048 has no dividend info
0048
Stock Code:4758 has no dividend info
475

In [7]:
div_info_final

,Announced,Financial Year,Subject,EX Date,Payment Date,Amount,Indicator,Stock Code,Year
0,2022-04-20,2021-12-31,Interim Dividend,2022-05-10,2022-05-27,0.0260,Currency,5250,2021
1,2021-04-15,2020-12-31,Interim Dividend,2021-05-05,2021-05-21,0.0160,Currency,5250,2020
2,2020-04-22,2019-12-31,Interim Dividend,2020-05-12,2020-05-28,0.0230,Currency,5250,2019
3,2019-04-29,2018-12-31,Interim Dividend,2019-05-09,2019-05-23,0.0240,Currency,5250,2018
4,2022-12-01,2022-12-31,Income Distribution,2022-12-23,2023-01-12,0.0385,Currency,0800EA,2022
...,...,...,...,...,...,...,...,...,...
4088,2022-05-26,2022-06-30,Interim Dividend,2022-06-10,2022-06-29,0.0200,Currency,7028,2022
4089,2021-09-08,2021-06-30,Second Interim Dividend,2021-09-23,2021-10-12,0.0250,Currency,7028,2021
4090,2021-05-28,2021-06-30,Interim Dividend,2021-06-14,2021-06-29,0.0200,Currency,7028,2021
4091,2019-08-29,2019-06-30,Interim Dividend,2019-10-25,2019-11-13,0.0500,Currency,7028,2019


In [8]:
filtered_df

,Announced,Financial Year,Subject,EX Date,Payment Date,Amount,Indicator,Stock Code,Year


In [11]:
div_info_final.to_csv('div_info_final.csv', mode='a', index=True, header=True)